Welcome to **Mosaic Art Creater**

このコードはGithubで[公開されています](https://github.com/Cherry-115/Mosaic-Art-Creator)  
これにより作成するモザイクアートの画像や完成品などが公開されることは**ありません**  
あくまで、コードのみを公開しています  
（コードを理解できる方はぜひ、IssueやPull Requestをしてください！）

作成手順  
<details>
  <summary>モザイクアートの設定</summary>
    下のスライダーなどを動かすことで各種項目の設定ができます  <br>
    <code>mosaic_resolution</code>は一つ一つのモザイクの解像度を指定することができます 初期値: 200px  <br>
    また、一つ一つのモザイクはすべて1:1のアスペクト比になります  <br>
    <code>target_width_resolution</code>は目標画像の<b>横</b>解像度を指定できます 初期値: 300  <br>
    <code>target_height_resolution</code>は目標画像の<b>縦</b>解像度を指定できます 初期値: 300  <br>
    <br>
    最終的な解像度は<code>mosaic_resolution</code> × <code>target_width_resolution</code> × <code>target_height_resolution</code>で決まります  <br>
    例えばモザイク画像が100pxでtargetが300 × 300だった場合最終解像度は30,000px × 30,000pxになります  <br>
    決定したら実行ボタンを押してください(一度押せば数値を変えても自動で計算してくれます)<br>
    <br>
    <code>use_manual_path</code>は、Google Drive等にあるファイルを使いたいときに手動でパスを指定することで   <br>
    そのファイルを使えるようにするものです   <br>
    基本的にこのチェックボックスと<code>target_image_path</code> <code>mosaic_image_path</code>は<b>使わないでください</b>   <br>
    <code>use_manual_path</code>にチェックを付けることで、この後の画像をアップロードする処理を飛ばし  <br>
    書いてあるパスから画像を読みます   <br>
    また、<code>mosaic_image_paths</code>はディレクトリのパスを指定してください<br>
    自動的にファイル内の画像を読み込みます<br>
</details>
<br>
<details><summary>モザイクアートの作成</summary>
    次に<b>モザイクアートの作成</b>ブロックにある実行ボタンを押します<br>

</details>







In [ ]:
# @title 設定 {display-mode: "form", run:"auto"}
#@markdown <= 実行ボタンを押してください
import glob
import sys
import os
from mimetypes import guess_type

#オブジェクト指向の言語なんだからクラスを使えよ　はい
class Settings:
    def __init__(self,
                 mosaic_resolution: int,
                 target_width_resolution: int,
                 target_height_resolution: int,
                 use_manual_path: bool,
                 target_image_manual_path: None | str,
                 mosaic_image_manual_paths: None | str,
                 ) -> None:
        self.MosaicResolution = mosaic_resolution
        self.TargetWidthResolution = target_width_resolution
        self.TargetHeightResolution = target_height_resolution
        self.UseManualPath = use_manual_path

        if not target_image_manual_path:
            self.TargetImageManualPath = None

        if not mosaic_image_manual_paths:
            self.MosaicImageManualPaths = None

        self.TargetImageManualPath = target_image_manual_path
        self.MosaicImageManualPaths = mosaic_image_manual_paths

    def CheckSettings(self):
        if self.UseManualPath:
            if not self.TargetImageManualPath:
                raise ValueError("Target image path is empty")

            if not self.MosaicImageManualPaths:
                raise ValueError("Mosaic image path is empty")

            if not os.path.isfile(self.TargetImageManualPath):
                raise ValueError("Target image path is not found")

            if not os.path.isdir(self.MosaicImageManualPaths):
                raise ValueError("Mosaic image path is not found")

            if guess_type(self.TargetImageManualPath)[0] != "image/jpeg" and guess_type(self.TargetImageManualPath)[0] != "image/png":
                raise ValueError("Target image is must be jpg or png")

            for d in glob.glob(self.MosaicImageManualPaths + "/*"):
                if guess_type(d)[0] != "image/jpeg" and guess_type(d)[0] != "image/png":
                    raise ValueError(f"""Found files that cannot be used as is\n
                                     {d}\n
                                     Tip: Please make sure that all the contents of the folder containing the material images are jpg or png images!""")

            print("All settings are correct")
            print("↓Final resolution\nheight: {0}\nwidth: {1}".format(str(mosaic_resolution * target_height_resolution), str(mosaic_resolution * target_width_resolution)))

    def GetMosaicResolution(self) -> int:
        """
        モザイク画像の解像度を取得します
        1:1のため、int型で返します
        """
        return self.MosaicResolution

    def GetTargetResolution(self) -> tuple(int, int):
        """
        ターゲット画像の解像度を取得します
        (Width, Height)
        """
        return (self.TargetWidthResolution, self.TargetHeightResolution)

    def GetUseManualPath(self) -> bool:
        """
        手動でパスを入力するかどうかを取得します
        """
        return self.UseManualPath

    def GetTargetImageManualPath(self) -> str:
        """
        ターゲット画像のパスを取得します
        """
        return self.TargetImageManualPath

    def GetMosaicImageManualPaths(self) -> str:
        """
        モザイク画像のパスを取得します
        """
        return self.MosaicImageManualPaths




mosaic_resolution = 200 #@param {type: "slider", min: 10, max: 1000, step: 10}
target_width_resolution = 27 #@param {type: "slider", min: 10, max: 1000, step: 1}
target_height_resolution = 59 #@param {type: "slider", min: 10, max: 1000, step: 1}
use_manual_path = False #@param {type: "boolean"}
target_image_manual_path = "" #@param {type: "string"}
mosaic_image_manual_paths = "" #@param {type: "string"}

settings = Settings(mosaic_resolution, target_width_resolution, target_height_resolution, use_manual_path, target_image_manual_path, mosaic_image_manual_paths)

settings.CheckSettings()

In [ ]:
#@title モザイクアートの作成 {display-mode: "form"}
#@markdown <= 実行ボタンを押してください
from PIL import Image
import glob
import numpy as np
import cv2
import os
from random import randint
import time
from IPython.utils import io
import subprocess
from tqdm.notebook import tqdm
import tempfile
from google.colab import files
import sys
import shutil

class Color:
    BLACK = "\033[30m"
    RED = "\033[31m"
    GREEN = "\033[32m"
    YELLOW = "\033[33m"
    BLUE = "\033[34m"
    WHITE = "\033[37m"
    END = "\033[39m" # 文字色のみの終了コード
class Decorate:
    BOLD = "\033[1m"
    UNDERLINE = "\033[4m"
    INVISIBLE = "\033[08m"
    REVERSE = "\033[07m"
    END = "\033[0m" # 全ての色指定や装飾の終了コード
class BgColor:
    BLACK = "\033[40m"
    RED = "\033[41m"
    GREEN = "\033[42m"
    YELLOW = "\033[43m"
    BLUE = "\033[44m"
    WHITE = "\033[47m"
    END = "\033[49m" # 背景色のみの終了コード


class PrintPlus:
    def __init__(self, *args, **kwargs):
        self.args = args
        self.kwargs = kwargs

    def info(self, text: str):
        print(f"""{Color.GREEN}INFO{Color.END}  {text}""")

    def error(self, text: str):
        print(f"""{BgColor.RED}ERROR{BgColor.END}  {text}""")

    def warning(self, text: str):
        print(f"""{Color.YELLOW}WARNING{Color.END}  {text}""")

    def success(self, text: str):
        print(f"""{Color.BLUE}SUCCESS{Color.END}  {text}""")

    def bold(self, text: str):
        print(f"""{Decorate.BOLD}{text}{Decorate.END}""")

    def underline(self, text: str):
        print(f"""{Decorate.UNDERLINE}{text}{Decorate.END}""")

    def invisible(self, text: str):
        print(f"""{Decorate.INVISIBLE}{text}{Decorate.END}""")


def upload_mosaic_file() -> list:
    log.underline("↓Click Here!")
    mosaic_image_paths = list(files.upload().keys())

    if any(mosaic_image_paths) != True:
        log.error("This is None type. Do you cancel Upload?")
        raise FileNotFoundError("No file. Please try again.")

    for path in mosaic_image_paths:
        if path[-3:] == 'jpg' or path[-3:] == 'png':
            #print('File Path:' + path)
            pass
        else:
            raise TypeError('Need for png or jpg')
    log.info("Upload complete for Mosaic Photos")
    return mosaic_image_paths

def upload_target_file() -> str:
    log.underline("↓Click Here!")
    target_image_path = list(files.upload().keys())

    if any(target_image_path) != True:
        log.error("This is None type. Do you cancel Upload?")
        raise FileNotFoundError("No file. Please try again.")

    if len(target_image_path) == 1:
        pass
    else:
        raise ValueError('Please upload for ONE file')

    if target_image_path[0][-3:] == 'jpg' or target_image_path[0][-3:] == 'png':
        log.info("Upload complete for Target File")
        return target_image_path[0]
    else:
        raise TypeError('Need for png or jpg')

def apply_transparent_color_filter(image, RGB: (int, int, int, int), alpha):
    # Convert image to 'RGBA' if not already
    if image.mode != "RGBA":
        image = image.convert("RGBA")

    rgbA = (RGB[0], RGB[1], RGB[2], alpha)
    # print(rgbA)

    # 半透明の赤色フィルターを作成
    red_filter = Image.new("RGBA", image.size, rgbA)  # 最後の128は透明度を表す

    # フィルターを適用
    img_with_filter = Image.alpha_composite(image, red_filter)

    return img_with_filter


def create_base_image(size, color):
    # 新しい画像を作成
    return Image.new("RGBA", size, color)


def place_images(base_image, images_to_place_paths, positions):
    # 各位置に画像を配置
    position_width, position_height, _ = positions.shape

    place_bar = tqdm(total=position_width * position_height, bar_format=tqdm_BAR_FORMAT, position=1)
    place_bar.set_description("Placing images")

    for i, row in enumerate(positions):
        for j, rgb in enumerate(row):
            k = randint(0, len(images_to_place_paths) - 1)

            image_to_place = Image.open(images_to_place_paths[k])
            filterd_image = apply_transparent_color_filter(
                image_to_place, rgb, 200
            )
            adjusted_position = (
                j * filterd_image.width,
                i * filterd_image.height,
            )
            base_image.paste(filterd_image, adjusted_position)
            place_bar.update()

    # 結果を保存
    log.info("Exporting")
    base_image.save(r"/content/Mosaic_art.png")
    files.download(r"/content/Mosaic_art.png")


def image_to_matrix(image_path, is_path=True):
    if is_path:
        # 画像を読み込む
        img = Image.open(image_path)
    else:
        img = image_path
    # 画像をRGB形式に変換
    img_rgb = img.convert("RGB")
    # 画像のピクセルの色をNumPyの行列に変換
    matrix = np.array(img_rgb)
    return matrix


def Create_temp_image(image_paths, temp_dirpath) -> list:
    file_name_index = 0
    temp_image_paths = []
    # 画像ごとに処理を行う

    bar = tqdm(total=len(image_paths), bar_format=tqdm_BAR_FORMAT, position=0)
    bar.set_description("Creating temp images")

    for image in image_paths:
        try:
            # 画像を読み込む
                img = cv2.imread(image)
                # 画像の高さと幅を取得
                height, width = img.shape[:2]
                # アスペクト比を1:1にするために、短辺に合わせて余分な箇所を切り取る
                if height > width:
                    # 縦長の場合、上下を切り取る
                    crop = img[(height - width) // 2 : (height + width) // 2, :]

                    # 横長の場合、左右を切り取る
                    crop = img[:, (width - height) // 2 : (width + height) // 2]
                else:
                    # 正方形の場合、そのまま
                    crop = img
                # 画像の解像度を200*200に縮小する
                resize = cv2.resize(crop, (mosaic_resolution, mosaic_resolution))
                # 画像を保存する
                path = temp_dirpath + "resized_" + str(file_name_index) + ".png"
                cv2.imwrite(path, resize,)
                temp_image_paths.append(path)
                # print("resized_" + str(file_name_index) + ".png")

                bar.update()

                file_name_index += 1
        except AttributeError as e:
            _, _, exc_tb = sys.exc_info()
            log.error(str(e) + " At line: " + str(exc_tb.tb_lineno))
            file_name_index += 1
            pass
    return temp_image_paths



def Delete_temp(temp_path):
    shutil.rmtree(temp_path)


def resize_image(input_path, new_width, new_height) -> Image:
    # 画像を開く
    original_image = Image.open(input_path)

    if original_image.width <= new_width or original_image.height <= new_height:
        raise Exception("The image is smaller than the specified size")
    else:
        return Image.open(input_path).resize((new_width, new_height))

def Create_temp_dir():
    return tempfile.mkdtemp()

if __name__ == "__main__":
    log = PrintPlus()
    log.bold("Welcome to Mosaic Art Creater")
    log.info(f"""Settings\n
             mosaic_resolution => {str(mosaic_resolution)}px\n
             target_width_resolution => {str(target_width_resolution)}px\n
             target_height_resolution => {str(target_height_resolution)}px\n
             target_image_manual_path => {str(target_image_manual_path) if use_manual_path else "None"}\n
             mosaic_image_manual_paths => {str(mosaic_image_manual_paths) if use_manual_path else "None"}""")

    if use_manual_path == False:
        target_image_path = upload_target_file()
        mosaic_image_paths = upload_mosaic_file()
    else:
        print(f"""{Color.GREEN}INFO{Color.END}  You are turn on for {Decorate.BOLD}use_manual_path{Decorate.END} Because so, Using manual input paths""")

    tqdm_BAR_FORMAT = (
        "{l_bar}{bar}| {n_fmt}/{total_fmt} [elapsed: {elapsed} remaining: {remaining}]"
    )
    log.info("Number of images: {}".format(len(mosaic_image_paths)))
    temp_dirpath = Create_temp_dir()
    log.info("Creating temp images")
    temp_image_paths = Create_temp_image(mosaic_image_paths, temp_dirpath)
    log.info("Resizing Target image")
    target_image = resize_image(target_image_path, target_width_resolution, target_height_resolution)
    positions = image_to_matrix(target_image, is_path=False)  # 画像を配置する位置
    #   [[[R,G,B],[R,G,B]],
    #    [[R,G,B],[R,G,B]],
    #    [[R,G,B],[R,G,B]],]
    #
    #
    #

    if len(mosaic_image_paths) < positions.shape[0] * positions.shape[1]:
        log.info("画像の数が足りないため、画像を使いまわします")
    position_height, position_width, _ = positions.shape
    # print(positions)
    #print(positions.shape)
    #print(position_width)
    #print(position_height)

    # 出力解像度を使用する画像の合計に設定
    total_size = (position_width * mosaic_resolution, position_height * mosaic_resolution)
    log.info('最終出力サイズ：' + str(total_size))

    color = (0, 0, 0)  # ベース画像の色
    base_image = create_base_image(total_size, color)
    log.info("Creating MosaicArt")
    place_images(
        base_image, temp_image_paths, positions
    )

    Delete_temp(temp_dirpath)
    log.info("Cleaning temp")
    log.success("MozaicArt Complete!")